<a href="https://colab.research.google.com/github/LoloziAnas/deeplearning/blob/master/PredictingThePriceOfUsedCars_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


In [131]:
# Import the dataset.
dataset = pd.read_csv('toyota.csv')
dataset.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,Manual,24089,Petrol,265,36.2,2.0
1,GT86,2017,15995,Manual,18615,Petrol,145,36.2,2.0
2,GT86,2015,13998,Manual,27469,Petrol,265,36.2,2.0
3,GT86,2017,18998,Manual,14736,Petrol,150,36.2,2.0
4,GT86,2017,17498,Manual,36284,Petrol,145,36.2,2.0


In [132]:
# Cleaning the data.

print(dataset['transmission'].unique())
# output -> ['Manual' 'Automatic' 'Semi-Auto' 'Other']

#Exclude the case that contains Other.
dataset = dataset[dataset['transmission'] != 'Other']

print(dataset['fuelType'].unique())
# output -> ['Petrol' 'Other' 'Hybrid' 'Diesel']

#Exclude the case that contains Other.
dataset = dataset[dataset['fuelType'] != 'Other']



['Manual' 'Automatic' 'Semi-Auto' 'Other']
['Petrol' 'Other' 'Hybrid' 'Diesel']


In [133]:
from sklearn.preprocessing import LabelEncoder

lEnc = LabelEncoder()

dataset['transmission'] = lEnc.fit_transform(dataset['transmission'])
dataset['fuelType'] = lEnc.fit_transform(dataset['fuelType'])

dataset

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,GT86,2016,16000,1,24089,2,265,36.2,2.0
1,GT86,2017,15995,1,18615,2,145,36.2,2.0
2,GT86,2015,13998,1,27469,2,265,36.2,2.0
3,GT86,2017,18998,1,14736,2,150,36.2,2.0
4,GT86,2017,17498,1,36284,2,145,36.2,2.0
...,...,...,...,...,...,...,...,...,...
6733,IQ,2011,5500,0,30000,2,20,58.9,1.0
6734,Urban Cruiser,2011,4985,1,36154,2,125,50.4,1.3
6735,Urban Cruiser,2012,4995,1,46000,0,125,57.6,1.4
6736,Urban Cruiser,2011,3995,1,60700,2,125,50.4,1.3


In [134]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformcolumn = ColumnTransformer([("transmission",OneHotEncoder(dtype=int, ),['transmission','fuelType'])],remainder='passthrough')

dataset = transformcolumn.fit_transform(dataset)
pd.DataFrame(dataset)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1,0,0,0,1,GT86,2016,16000,24089,265,36.2,2
1,0,1,0,0,0,1,GT86,2017,15995,18615,145,36.2,2
2,0,1,0,0,0,1,GT86,2015,13998,27469,265,36.2,2
3,0,1,0,0,0,1,GT86,2017,18998,14736,150,36.2,2
4,0,1,0,0,0,1,GT86,2017,17498,36284,145,36.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6627,1,0,0,0,0,1,IQ,2011,5500,30000,20,58.9,1
6628,0,1,0,0,0,1,Urban Cruiser,2011,4985,36154,125,50.4,1.3
6629,0,1,0,1,0,0,Urban Cruiser,2012,4995,46000,125,57.6,1.4
6630,0,1,0,0,0,1,Urban Cruiser,2011,3995,60700,125,50.4,1.3


In [152]:
# Splitting data to features and target.
x = dataset[:,[i for i in range(0,13) if i != 8 and i != 6 ]]
y = dataset[:,8]
x.shape

(6632, 11)

In [153]:
# Splitting data into training and test data.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [154]:

from sklearn import svm
from sklearn.model_selection import GridSearchCV

clf = svm.SVR()
# defining parameter range 
param_grid = {'C': [ i for i in range(1,10)],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'epsilon':[i*0.1 for i  in range(1,10)],'kernel': ['rbf']}  

parameters = {'C': [1000],'gamma': [0.001],'epsilon':[0.1],'kernel': ['rbf']}  

paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}

regr = GridSearchCV(clf,parameters, refit = True, verbose = 1)

# fitting the model for grid search 
%time regr.fit(x_train,y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    7.0s finished


CPU times: user 8.85 s, sys: 145 ms, total: 8.99 s
Wall time: 9 s


GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1000], 'epsilon': [0.1], 'gamma': [0.001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [ ]:
#**Using RandomizedSearchCV**

#Same parameter space, but only test 250 random combinations.
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

regr = RandomizedSearchCV(clf,
                        param_distributions=paramgrid,
                        n_iter=250,
                        cv=StratifiedKFold(n_splits=2),
                        verbose=1)
regr.fit(x_train,y_train)



In [155]:

# print best parameter after tuning 
print(regr.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(regr.best_estimator_) 


{'C': 1000, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


In [156]:
pred_y_ = regr.predict(x_test)
pred_y_
y_test

regr.score(x_train,y_train)

0.13772594923593418

In [161]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

print("R2-score: %.2f" % r2_score(pred_y_ , y_test) )
print(" (MSE): %.2f" % mean_squared_error(y_test, pred_y_))
print("Mean absolute error: %.2f" % np.mean(np.absolute(pred_y_ - y_test)))


R2-score: -45.72
 (MSE): 41414811.02
Mean absolute error: 4389.14
